<a href="https://colab.research.google.com/github/mattbarrett98/mykit-learn/blob/update/mklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# fast linear algebra, useful for every algorithm
import numpy as np

# for minimising logistic loss and fast computation of log sum exponential
import scipy.optimize
from scipy.special import logsumexp 

# to create abstract base class for all classifiers
from abc import ABCMeta, abstractmethod

# to speed up algorithm for support vector classifier
%load_ext Cython

# to parallelise random forest
import multiprocessing as mp

# Cython code for support vector classifier

In [2]:
%%cython
cimport numpy as np
import numpy as np
cimport cython
from scipy.linalg.cython_blas cimport ddot
from libc.stdlib cimport rand, srand
from libc.time cimport time
srand(time(NULL))

"""This is our implementation of the sequential minimal optimisation algorithm
(SMO) which we will use for the support vector classifier (SVC). It is defined 
here (and not in the block below with the rest of the code) because we have 
written it in Cython for performance improvement.
"""

cdef int mi_rand_int(int upper, int i):
    """Gives us a random integer in the range [0, upper] not equal to i."""
    cdef int j = i 
    while(j == i):
        j = rand() % (upper)
    return j  

cdef double mi_dot(np.ndarray[double, ndim=1] a, np.ndarray[double, ndim=1] b):
    """We use the blas function ddot, which returns the dot product of two 
    1D arrays containing double precision values. This function is slightly 
    faster than numpy's dot(). 
    """ 
    cdef int n = a.shape[0]
    cdef int stride = 1
    return ddot(&n, &a[0], &stride, &b[0], &stride)

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef SMO(np.ndarray[double, ndim=2] X, np.ndarray[double, ndim=1] y, double C, 
         double epsilon, double tol, int max_iter):
        """This is where SMO is carried out. The implementation and notation 
        used is consistent with http://cs229.stanford.edu/materials/smo.pdf . 
        As well as using blas' ddot instead of np.dot, we use a few other tricks
        to speed up the code. 
        
        - We statically type all the variables e.g. 'cdef int count' specifies 
        that the variable 'count' is an integer.

        - We disable bounds checking (decorators above the function). So each
        time we index an array there aren't any checks for out of bounds or
        negative indices. 

        These changes result in a 35% increase in speed compared to my original
        python implementation. 

        Parameters
        ----------
        X : array of type float64 and shape (n_training_obs, n_training_obs). In
        our implementation this is the radial basis function (RBF) kernel 
        applied to the training data.

        y : array of type float64 and shape (n_training_obs, ). Contains the
        true class of each training observation.  

        C : float64. Regularisation parameter. 

        epsilon : float64. If the norm of the difference in the alpha parameters
        from one iteration to the next is less than epsilon, the algorithm
        terminates.

        tol : float64. Tolerance- controls how small the error needs to be to 
        consider the corresponding alpha parameter optimal. 

        max_iter : int. Maximum number of iterations to perform. 

        Returns
        -------
        alpha : array of type float64 and shape (n_training_obs, ). Contains the
        values of the alpha parameters which minimise our SVC loss.   

        b : float64. The value of b in our optimal solution.    
        """
        cdef int count = 0
        cdef Py_ssize_t i, j, n
        cdef double L, H, b, b1, b2, alpha_j, alpha_i, E_i, E_j, eta
        n = X.shape[0]
        cdef np.ndarray[double, ndim=1] diff
        cdef np.ndarray[double, ndim=1] alpha = np.zeros((n))
        b = 0
        while True:
            count += 1
            alpha_prev = np.copy(alpha)
            for i in range(n):
                alpha_y = alpha * y
                E_i = mi_dot(X[i, :], alpha_y) + b - y[i]
                # if alpha_i doesn't satisfy optimal conditions then we try to 
                # optimise it 
                if ((E_i*y[i]<-tol and alpha[i]<C) or 
                   (E_i*y[i]>tol and alpha[i]>0)):  
                    
                    j = mi_rand_int(n, i)
                    eta = 2*X[i,j] - 2
                    if eta == 0:
                        continue
                    alpha_j, alpha_i = alpha[j], alpha[i]
                    if y[i] != y[j]:
                        L = max(0, alpha_j - alpha_i)
                        H = min(C, C - alpha_i + alpha_j)
                    else:
                        L = max(0, alpha_i + alpha_j - C)
                        H = min(C, alpha_i + alpha_j)

                    E_j = mi_dot(X[j,:], alpha_y) + b - y[j]
                    alpha[j] = alpha_j - y[j]*(E_i - E_j)/eta
                    alpha[j] = max(alpha[j], L)
                    alpha[j] = min(alpha[j], H)
                    alpha[i] = alpha_i + y[i]*y[j]*(alpha_j - alpha[j])

                    b1 = b - E_i - y[i]*(alpha[i] - alpha_i) \
                                 - y[j]*(alpha[j] - alpha_j)*X[i,j]
                    
                    b2 = b - E_j - y[j]*(alpha[j] - alpha_j) \
                                 - y[i]*(alpha[i] - alpha_i)*X[i,j]
                    if alpha[i]>0 and alpha[i]<C:
                        b = b1
                    elif alpha[j]>0 and alpha[j]<C:
                        b = b2
                    else: 
                        b = (b1 + b2)/2    

            # Check convergence
            diff = alpha - alpha_prev
            if mi_dot(diff, diff) < epsilon:
                break

            if count >= max_iter:
                return alpha, b
        
        return alpha, b

# Base classes

In [7]:
class BaseClassifier:
    """Base classifier from which all our classifiers will inherit since they
    all have some shared functionality. They each have a classification accuracy
    associated with them and need a method to calculate that accuracy. 

    We also have a magic method to allow us to compare our implementation to 
    sklearn's. We consider the implementations to be equivalent if their
    classification accuracies are within 0.5% of each other. 
    """
    def __init__(self):
        self.accuracy = None

    def evaluate_predictions(self, predictions, true_classes):
        n_correct_predictions = sum(predictions == true_classes)
        n_predictions = predictions.shape[0]
        self.accuracy = 100 * n_correct_predictions/n_predictions
        return

    def __eq__(self, sklearn_model):
        diff = self.accuracy - sklearn_model.accuracy
        diff = round(diff, 2)
        if diff == 0:
            return "True, mklearn's accuracy is the same as sklearn's."
        if diff > 0 and diff <= 0.5:
            return "True, mklearn's accuracy is just {}% higher than sklearn's.".format(diff)
        if diff > 0 and diff > 0.5:
            return "False, mklearn's accuracy is {}% better than sklearn's.".format(diff)   
        
        if diff < 0 and diff >= -0.5:
                return "True, mklearn's accuracy is just {}% lower than sklearn's.".format(-diff)  
        if diff < 0 and diff < -0.5:
                return "False, mklearn's accuracy is {}% lower than sklearn's.".format(-diff)


class AbstractClassifier(metaclass=ABCMeta):
    """Abstract base class which ensures that all of our classifiers have
    a fit method and a predict method. Not much purpose to this other than 
    learning about metaclasses. 
    """
    @abstractmethod
    def fit():
        pass

    @abstractmethod
    def predict():
        pass   

# Preprocessing

In [ ]:
class Preprocessing:
    """Class containing all necessary methods to get our data into the 
    correct format.

    - flatten_image: converts nxn images into 1D arrays of length n**2.

    - normalise_data: normalises all pixel values into [0,1].

    - one_hot_encode: converts class value to an array of zeros, with a 1 at
    the index corresponding to the class value e.g. 2->(0,0,1,0,0,0,0,0,0,0).

    - get_svc_output: converts the class value to either +1 or -1. For 
    example, for the SVC for class '0' the function replaces any 0 in 
    train_y with +1 and -1 for any other class.  
    """

    @staticmethod
    def flatten_image(train_data, test_data):
        train_flattened = train_data.reshape(train_data.shape[0], -1)
        test_flattened = test_data.reshape(test_data.shape[0], -1)
        return train_flattened, test_flattened

    @staticmethod
    def normalise_data(train_data, test_data, max_value=255):
        return train_data/max_value, test_data/max_value

    @staticmethod
    def one_hot_encode(classes):
        one_hot_encoding = np.zeros((classes.shape[0], len(set(classes))))
        for i in range(len(one_hot_encoding)):
            one_hot_encoding[i, classes[i]] = 1
        return one_hot_encoding   

    @staticmethod
    def get_svc_output(train_y):
        train_y_svc = -1 * np.ones((len(set(train_y)), train_y.shape[0]))
        for j in range(train_y_svc.shape[0]):
            for i in range(train_y_svc.shape[1]):
                if train_y[i] == j:
                    train_y_svc[j, i] = 1
        return train_y_svc     

# K nearest neighbours

In [ ]:
class MiKNeighboursClassifier(BaseClassifier, AbstractClassifier):
    """Classifier implementing the k-nearest-neighbours algorithm. The
    algorithm works by finding the k training observations which are
    closest to a test point in euclidean distance. The predicted class
    of this test point is then given by the most frequent occurring class
    of these k training points.

    Attributes
    ----------
    k : int. The 'k' in k-nearest-neighbours: the number of neighbours to
    use in the algorithm.

    train_x : numpy array of shape (n_training_obs, n_features) to store
    the training data.

    train_y : numpy array of shape (n_training_obs, ) to store the true
    class of each training observation.

    n_test_obs : int. Number of test observations. 

    n_splits : int. The number of groups we want to break the test data
    into.

    split_size : int. The number of points in each split of the test data. 
    """

    def __init__(self, k, n_splits):
        self.k = k
        self.train_x = None
        self.train_y = None
        self.n_test_obs = None
        self.n_splits = n_splits
        self.split_size = None

    def fit(self, train_x, train_y):
        """In KNN all the calculations are done once we have the test data.
        Hence all we need to do here is store the training data. 
        """
        self.train_x = train_x
        self.train_y = train_y

    def _find_euclidean_distances(self, test_x):
        """This is a generator which returns the euclidean distances between
        the training points and the ith split of test points. The reason we
        split the test points up is that calculating the euclidean distance
        between every training and test point in one go would use too much
        memory.

        The squared euclidean distance between vectors x and y is given by
        (x - y).T * (x - y) which we can rewrite as x.T*x - 2x.T*y + y.T*y.
        This formula allows us to calculate the euclidean distance between
        every pair of vectors in the training data and the test data split
        efficiently. It also has the added bonus that we only need to
        compute x.T*x once, where x is the training data.

        Also note that we are working with the squared euclidean distances,
        since finding the k smallest euclidean distances is equivalent to
        finding the k smallest squared euclidean distances.

        Parameters
        ----------
        test_x : numpy array of shape (n_test_samples, n_features) containing
        the test data.
        """
        train_x_squared_norms = (self.train_x * self.train_x).sum(axis=1)
        for i in range(self.n_splits):
            # get ith split of test points and find euc distances
            lower_idx = self.split_size * i
            upper_idx = self.split_size * (i + 1)
            test_i = test_x[lower_idx: upper_idx]
            euc_dists = -2 * np.matmul(self.train_x, test_i.T)
            euc_dists += train_x_squared_norms[:, np.newaxis]
            euc_dists += (test_i * test_i).sum(axis=1).T[np.newaxis, :]
            yield euc_dists

    def predict(self, test_x):
        """This returns the predicted classes of the test data given by knn.

        Parameters 
        ----------
        test_x : numpy array of shape (n_test_samples, n_features) containing
        the test data.
        """
        self.n_test_obs = test_x.shape[0]
        self.split_size = int(self.n_test_obs / self.n_splits)
        preds = np.empty(self.n_test_obs)
        euc_dist_generator = self._find_euclidean_distances(test_x)
        for i in range(self.n_splits):
            euc_distances = next(euc_dist_generator)
            k_neighbours = np.argsort(euc_distances, axis=0)[0:self.k, :]
            k_classes = self.train_y[k_neighbours]
            class_counts = [np.bincount(k_classes[:, i], minlength=10)
                            for i in range(self.split_size)]
            class_counts_arr = np.asarray(class_counts)
            lower_idx = self.split_size * i
            upper_idx = self.split_size * (i + 1)
            preds[lower_idx:upper_idx] = np.argmax(class_counts_arr, axis=1)
        return preds

# Multinomial logistic regression

In [ ]:
class MiLogisticClassifier(BaseClassifier, AbstractClassifier):
    """Classifier implementing multinomial logistic regression.

    The classifier assumes each of the training point classes is 
    categorically distibuted with the class probabilites given by the 
    multinomial logistic model. To estimate the parameters of the model, 
    we will use Bayesian statistics. We set a standard normal prior on each
    parameter and use the training data to find the posterior distribution
    (or at least a function proportional to it). We will take the maximiser 
    of the posterior as our parameter estimates, known as the maximum a 
    posteriori (MAP). 

    Note that finding the maximum of the posterior is equivalent to finding 
    the minimum of the negative of the log posterior. So for numerical
    reasons we work with this function instead.
    """

    def __init__(self):
        self.n_features = None
        self.n_classes = None
        self.MAP = None

    def _neg_log_posterior_grad(self, betas, train_x, train_y):
        """We compute the gradient of the negative log posterior in order to
        speed up convergence of the minimisation. 

        Parameters
        ----------
        betas : a vector of parameters of length (n_features+1) * n_classes,
        since there is 1 parameter for each feature plus an intercept term,
        for each class. 

        train_x : numpy array of shape (n_training_obs, n_features)
        containing the training data.

        train_y : numpy array of shape (n_training_obs, n_classes) 
        containing the one hot encoded true class of each training 
        observation.

        Returns
        -------
        neg_log_posterior : float. The value of the negative log posterior 
        distribution evaluated at betas.

        gradients.ravel() : flattened vector containing the gradients of the
        negative log posterior, with respect to each of the parameters. 
        """

        # calculate value of negative log posterior for betas
        beta_matrix = betas.reshape(self.n_classes, self.n_features + 1)
        weights = beta_matrix[:, :-1]
        intercepts = beta_matrix[:, -1]
        p = np.matmul(train_x, weights.T)
        p += intercepts
        p -= logsumexp(p, axis=1).reshape(p.shape[0], 1)
        neg_log_likelihood = -np.sum(p * train_y)
        neg_log_prior = 0.5 * np.sum(weights ** 2)
        neg_log_posterior = neg_log_likelihood + neg_log_prior

        # calculate gradients
        gradients = np.zeros((self.n_classes, self.n_features + 1))
        p = np.exp(p)
        diff = p - train_y
        gradients[:, 0:self.n_features] = np.matmul(diff.T, train_x)
        gradients[:, 0:self.n_features] += weights
        gradients[:, -1] = diff.sum(axis=0)
        return neg_log_posterior, gradients.ravel()

    def fit(self, train_x, train_y):
        """Here we minimise the negative log posterior (using L-BFGS-B) in 
        order to find our MAP estimates. It assigns these estimates to the 
        MAP attribute.

        Parameters
        ----------
        train_x : numpy array of shape (n_training_obs, n_features)
        containing the training data.

        train_y : numpy array of shape (n_training_obs, n_classes) 
        containing the one hot encoded true class of each training 
        observation.
        """
        self.n_features = train_x.shape[1]
        self.n_classes = train_y.shape[1]
        n_parameters = (self.n_features + 1) * self.n_classes
        optim = scipy.optimize.minimize(self._neg_log_posterior_grad,
                                        np.random.normal(size=n_parameters),
                                        args=(train_x, train_y),
                                        jac='TRUE',
                                        method='L-BFGS-B',
                                        options={'maxiter': 750}
                                        )
        self.MAP = optim.x.reshape(self.n_classes, self.n_features + 1)

    def predict(self, test_x):
        """Using our MAP estimates we calculate the class probabilities for 
        our test data. We take the class with the largest probability to be
        the predicted output. 

        Parameters
        ----------
        test_x : numpy array of shape (n_test_obs, n_features) containing 
        the test data.
        """
        weights_map = self.MAP[:, :-1].T
        intercepts_map = self.MAP[:, -1]
        class_prob_pred = np.matmul(test_x, weights_map) + intercepts_map
        predictions = np.argmax(class_prob_pred, axis=1)
        return predictions 

# Support vector classifier

In [ ]:
class MiSupportVectorClassifier(BaseClassifier, AbstractClassifier):
    """Implements a support vector classifier. An SVC is used for binary 
    classification problems. It first transforms the training data using
    some kernel function, in our case this is the Radial basis function 
    (RBF) kernel. Then in this transformed space, the SVC finds the 
    hyperplane which best separates the training points for the 2 classes. 
    This hyperplane can then be used to classify a new test point. 

    Attributes
    ----------
    C : float. Regularisation parameter. 

    epsilon : float. If the norm of the difference in the alpha parameters
    from one iteration to the next is less than epsilon, the SMO algorithm
    terminates.

    tol : float. Tolerance- controls how small the error needs to be to 
    consider the corresponding alpha parameter optimal. 

    max_iter : int. Maximum number of iterations to perform. 

    gamma : float. The scale to use in the calculation of the RBF kernel.

    alphas : This is where the optimal values of alpha from SMO are stored.

    b : This is where the optimal values of b from SMO are stored. 

    train_y : To store the training classes since it is needed in the
    predict method. 

    train_x : Stores the training data also required in predict method.
    """

    def __init__(self, C, epsilon, tol, max_iter):
        self.C = C
        self.epsilon = epsilon
        self.tol = tol
        self.max_iter = max_iter
        self.gamma = None
        self.alphas = None
        self.b = None
        self.train_y = None
        self.train_x = None  

    def _compute_rbf_kernel(self, X, Y):
        """Takes 2 arrays and calculates the RBF kernel between every pair
        of vectors in each array. It calculates the squared Euclidean 
        distance between every pair of vectors between X and Y, which is an
        intermediate value needed to find the RBF kernels.

        Parameters
        ----------
        X, Y : Both 2D numpy arrays of floats. They can have any shape, 
        however their first axis sizes must match: X.shape[1] == Y.shape[1].  
        """
        euc_dists = -2 * np.matmul(X, Y.T)
        euc_dists += (X * X).sum(axis=1)[:, np.newaxis]
        euc_dists += (Y * Y).sum(axis=1).T[np.newaxis, :]
        np.fill_diagonal(euc_dists, 0)
        rbf_kernel = np.exp(-self.gamma * euc_dists)
        return rbf_kernel

    def fit(self, train_x, train_y):
        """Since SVCs perform binary classification and we have 10 classes 
        we must fit 10 different SVCs (one for each class).

        Parameters
        ----------
        train_x : numpy array of shape (n_training_obs, n_features) 
        and type float, contains the training data.

        train_y : numpy array of shape (n_classes, n_training_obs) which
        contains the true classes of the training data (for each of the
        n_classes SVCs). 
        """
        self.train_y = train_y
        self.train_x = train_x
        n_classifiers = train_y.shape[0]
        self.alphas = np.zeros((n_classifiers, train_x.shape[0]))
        self.b = np.zeros(n_classifiers)
        self.gamma = 1 / (train_x.shape[1] * train_x.var())
        rbf_train_x = self._compute_rbf_kernel(train_x, train_x)
        for i in range(n_classifiers):
            self.alphas[i, :], self.b[i] = SMO(rbf_train_x,
                                                train_y[i, :],
                                                self.C,
                                                self.epsilon,
                                                self.tol,
                                                self.max_iter)

    def predict(self, test_x):
        """Calculates the predicted outputs of each of the binary SVCs. The
        prediction for each test point is given by the highest (most
        confident) output. This is known as one-vs-all classification. 

        Parameters
        ----------
        test_x : numpy array of shape (n_test_obs, n_features) and type
        float. Contains the test points to make predictions on. 
        """
        rbf_test_x = self._compute_rbf_kernel(test_x, self.train_x)
        svc_outputs = np.matmul(self.train_y * self.alphas, rbf_test_x.T) \
                      + self.b[:, np.newaxis]
        predictions = np.argmax(svc_outputs, axis=0)
        return predictions  

# Decision tree

In [6]:
class MiDecisionTreeClassifier(BaseClassifier, AbstractClassifier):
    """Implements a classifier based on a decision tree. It works by asking 
    a series of questions about the data point. For example, is the value of
    the 10th feature greater or less than 0.5. Based on the answer to this 
    it would then ask another question, and then another up to a maximum of 
    max_depth questions. The predicted class of a point is given by the most
    frequent occurring class of the training points which followed the same 
    path down this 'tree' of questions. 

    In order to figure out what questions are best to ask, we use a measure
    called the gini impurity. It measures how well a question splits up the
    data, with a question that splits data perfectly having a gini impurity 
    of 0. We calculate the gini impurity of every possible way of splitting
    the data, and choose the question with the lowest gini impurity.  

    Attributes
    ----------
    max_depth : int, default=None (no restriction on tree depth). The
    maximum number of questions we will ask about a point. 

    min_samples_split : int, default=2. The minimum number of training
    points we need at any point in the tree in order to continue asking 
    questions. 

    min_gini : float, default=1e-5. If the gini impurity of a split is
    lower than min_gini, then we stop asking questions i.e. we consider the
    classes of the data to be pure enough to make good predictions. 

    tree : dict. This is where we store all our questions once we have
    fitted the tree. 
    """
        
    def __init__(self, max_depth=None, min_samples_split=2, min_gini=1e-5):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_gini = min_gini
        self.tree = None

    def _find_best_split(self, train_x, train_y):
        """Function which finds the best possible way to split the data. It
        calculates the gini impurity for every possible way of splitting
        train_x i.e. for every feature and every value of each feature which
        uniquely splits train_x. 

        Parameters
        ----------
        train_x : numpy array of shape (n_training_obs, n_features) and type
        float. Contains the training data. 

        train_y : numpy array of shape (n_training_obs, n_classes) which 
        contains the one hot encoded true classes of the training data. 

        Returns
        -------
        best_split_gini : float. The value of the gini impurity for the best
        split of the data. 

        split_feature : int. The feature that is used in the best split. 

        split_value : float. The value of split_feature that is used in the 
        best split e.g. the best split is obtained by asking the question
        is split_feature less than or equal to split_value. 
        """
        sorted_x = np.argsort(train_x, axis=0)
        n_features = train_x.shape[1]
        n_classes = train_y.shape[1]
        n_obs = train_x.shape[0]
        # count the number of appearances of each class that appear above
        # or below each split (we have n_obs-1 possible splits per feature)
        class_counts_below = np.empty((n_obs - 1, n_features, n_classes))
        class_counts_above = np.empty((n_obs - 1, n_features, n_classes))
        n_obs_below = np.arange(1, n_obs)[:, None]
        n_obs_above = np.arange(1, n_obs)[::-1][:, None]
        # class counts after the first split for each feature
        class_counts_below[0] = train_y[sorted_x[0, :]]
        class_counts_above[0] = np.sum(train_y[sorted_x[1:, ]], axis=0)
        # class counts for all subsequent possible splits
        for i in range(1, n_obs - 1):
            class_counts_below[i] = class_counts_below[i - 1] \
                                    + train_y[sorted_x[i]]
            class_counts_above[i] = class_counts_above[i - 1] \
                                    - train_y[sorted_x[i]]
        probabilities_below = class_counts_below / n_obs_below[:, :, None]
        probabilities_above = class_counts_above / n_obs_above[:, :, None]
        ginis_below = 1 - np.sum(probabilities_below ** 2, axis=2)
        ginis_above = 1 - np.sum(probabilities_above ** 2, axis=2)
        gini_impurities = (ginis_below*n_obs_below
                          + ginis_above*n_obs_above) / n_obs
        # smallest gini impurity is the best
        arg_best = np.argmin(gini_impurities)
        unravelled_args = np.unravel_index(arg_best, gini_impurities.shape)
        best_split_gini = gini_impurities[unravelled_args]
        split_feature = unravelled_args[1]
        split_idx = unravelled_args[0]
        feature_arg_sort = sorted_x[:, split_feature]
        feature_values = train_x[feature_arg_sort, split_feature]
        split_value = (feature_values[split_idx]
                        + feature_values[split_idx + 1]) / 2
        return best_split_gini, split_feature, split_value

    def _perform_split(self, train_x, train_y, feature, split_value):
        """Once we have found the best possible split, this function will
        actually split the training data up for us. 

        Parameters
        ----------
        train_x : numpy array of shape (n_training_obs, n_features) and type
        float. Contains the training data. 

        train_y : numpy array of shape (n_training_obs, n_classes) which 
        contains the one hot encoded true classes of the training data. 

        feature : int. Feature used to perform split.

        split_value : float. Value of the feature used to perform split. 

        Returns
        -------
        train_x_below, train_y_below : the points in train_x and train_y
        whose value of 'feature' is less than or equal to split_value.

        train_x_above, train_y_above : the points in train_x and train_y
        whose value of 'feature' is greater than split_value. 
        """
        idx = train_x[:, feature] <= split_value
        train_x_below = train_x[idx]
        train_x_above = train_x[~idx]
        train_y_below = train_y[idx]
        train_y_above = train_y[~idx]
        return train_x_below, train_x_above, train_y_below, train_y_above

    def _build_tree(self, X, y, count):
        """A function to recursively build our tree. As long as max_depth
        hasn't been reached, we have at least min_samples_split and the 
        impurity is greater than min_gini, then we continue splitting the
        data by calling this function again. If one of these conditions is
        violated we stop splitting the data and note the most frequently
        occurring class of the remaining data. 

        Parameters
        ----------
        X : 2D numpy array of type float and shape (n, n_features) where n
        is the number of training data points remaining at the current node.

        y : numpy array of shape (n, n_classes) which contains the one hot 
        encoded true classes of the remaining training data. 

        count : int. Keeps track of the tree depth. 

        Returns
        -------
        sub_tree : dict. Contains the questions that we have chosen and the 
        most frequent occurring class found at the end of any path through 
        the tree.
        """
        if self.max_depth is None:
            depth_cond = True
        else:
            if count < self.max_depth:
                depth_cond = True
            else:
                depth_cond = False
        if X.shape[0] >= self.min_samples_split:
            samples_cond = True
        else:
            samples_cond = False

        if depth_cond & samples_cond:
            gini, feature, split_value = self._find_best_split(X, y)
            if gini >= self.min_gini:
                count += 1
                x_below, x_above, y_below, y_above = self._perform_split(
                                                              X, 
                                                              y, 
                                                              feature, 
                                                              split_value)

                question = "{} <= {}".format(feature, split_value)
                sub_tree = {question: []}
                # we use recursion to build the tree
                next_node_below = self._build_tree(x_below, y_below, count)
                next_node_above = self._build_tree(x_above, y_above, count)
                sub_tree[question].append(next_node_below)
                sub_tree[question].append(next_node_above)
            else:
                return np.argmax(np.sum(y, axis=0))
        else:
            return np.argmax(np.sum(y, axis=0))
        return sub_tree 

    def fit(self, X, y):
        """Initialises tree depth to 0 and builds the tree using training
        data X and their one hot encoded true classes y. 
        """
        count = 0
        self.tree = self._build_tree(X, y, count) 
        return self.tree         

    def make_prediction(self, test_obs, tree):
        """Recursively answers questions in our dict until it reaches the
        end of the tree and outputs the prediction for that path. 

        Parameters
        ----------
        test_obs : numpy array of shape (n_features, ) and type float. The 
        test point that we want to make a prediction for. 

        tree : dict. Here we pass the fitted tree found in fit(). 
        """
        question = list(tree.keys())[0]
        if test_obs[int(question.split()[0])] <= float(question.split()[2]):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
        if not isinstance(answer, dict):
            return answer
        return self.make_prediction(test_obs, answer)

    def predict(self, test_x):
        """Makes prediction for each test data point. 
        
        Parameters
        ----------
        test_x : numpy array of shape (n_test_obs, n_features) and type
        float. 
        """
        n_test_obs = test_x.shape[0]
        predictions = np.empty(n_test_obs)
        for i in range(n_test_obs):
            predictions[i] = self.make_prediction(test_x[i, :], self.tree)
        return predictions   

# Random forest

In [5]:
class MiRandomForestClassifier(BaseClassifier, AbstractClassifier):
    """Implements a classifier based on the random forest algorithm. This
    algorithm is just an ensemble of decision trees, which is where the
    forest part comes from.The random part comes from how we fit each of
    the decision trees. For each tree we resample the training points with
    replacement to form a different set of training points (of the same size
    as the original). We also take a random subset of features with which
    we fit the tree. This randomness is what will allow our classifier to
    generalise better than a single decision tree, which is inherently prone
    to overfitting.  

    Attributes
    ----------
    n_trees : int, default=1. The number of decision trees to fit. 

    max_features : int, default=100. The number of features to use in each
    decision tree. 

    base_estimator : instance of DecisionTreeClassifier with parameters
    max_depth, min_samples_split and min_gini. 

    n_jobs : int, default=1. The number of parallel processes to run. Should 
    be set to the number of cpu cores on your machine.

    forests : this is where all of the fitted decision trees (and what 
    subset of features they used) is stored.  
    """

    def __init__(self, n_trees=100, max_features=100, max_depth=20,
                  min_samples_split=2, min_gini=1e-5, n_jobs=1):
        self.n_trees = n_trees
        self.max_features = max_features
        self.base_estimator = MiDecisionTreeClassifier(max_depth, 
                                                      min_samples_split, 
                                                      min_gini)
        self.n_jobs = n_jobs
        self.forests = None

    def _build_forest(self, X, y, n_trees):
        """Function to fit a number of decision trees to form a forest.
        Remember for each tree we use a different random selection of
        training points and features. 

        Parameters
        ----------
        X : 2D numpy array of shape (n_training_obs, n_features) and type
        float. Contains the training data.

        y : numpy array of shape (n_training_obs, n_classes) which contains
        the one hot encoded true classes of the training data. 

        n_trees : int. The number of trees to fit in this forest. 

        Returns
        -------
        trees : list of length n_trees containing dicts. Each dict being one
        of the fitted trees. 

        feature_subsets : list of length n_trees, containing numpy arrays of
        shape (max_features, ). Each array contains the subset of features
        that each of the trees has been fit on. 
        """
        trees = []
        feature_subsets = []
        for i in range(n_trees):
            new_features = np.random.choice(X.shape[1], self.max_features, 
                                            replace=False)
            new_obs = np.random.choice(X.shape[0], X.shape[0])
            new_X = X[new_obs, :][:, new_features]
            new_y = y[new_obs]
            trees.append(self.base_estimator.fit(new_X, new_y))
            feature_subsets.append(new_features)
        return trees, feature_subsets

    def fit(self, X, y):
        """We use the multiprocessing module to construct our forest in 
        parallel. For example, if we have n_jobs=2 and n_trees=100, then this
        function will construct 2 forests each of 50 trees and construct them
        in parallel. Note the use of a context manager in the 'with' statement.
        This ensures the pool of processes is closed regardless of whether the
        code inside errors. 

        Parameters
        ----------
        X : 2D numpy array of shape (n_training_obs, n_features) and type
        float. Contains the training data.

        y : numpy array of shape (n_training_obs, n_classes) which contains
        the one hot encoded true classes of the training data. 

        Returns
        -------
        self.forests : contains a list of the fitted trees and feature 
        subsets from each parallel process. 
        """
        n_trees_job = int(self.n_trees / self.n_jobs)
        with mp.Pool(processes=self.n_jobs) as pool:
            self.forests = pool.starmap(self._build_forest, 
                                        [(X, y, n_trees_job) 
                                        for _ in range(self.n_jobs)])
        return self.forests

    def _tree_prediction(self, test_x, tree):
        """Make prediction of every test observation for a single tree.

        Parameters
        ----------
        test_x : numpy array of shape (n_test_obs, n_features) and type
        float.

        tree : dict containing the fitted tree. 
        """
        n_test_obs = test_x.shape[0]
        predictions = np.empty(n_test_obs)
        for i in range(n_test_obs):
            predictions[i] = self.base_estimator.make_prediction(test_x[i], 
                                                                  tree)
        return predictions

    def _forest_prediction(self, test_x, trees, feature_subsets):
        """Gives the prediction of every test observation, for each tree in
        a forest. 

        Parameters
        ----------
        test_x : numpy array of shape (n_test_obs, n_features) and type
        float.

        trees : list of dicts containing the fitted trees in the forest.

        feature_subsets : list of numpy arrays. Each array contains the subset
        of features used in each of the trees. 
        """
        n_trees_job = int(self.n_trees / self.n_jobs)
        predictions = np.empty((n_trees_job, test_x.shape[0]))
        for i in range(n_trees_job):
            predictions[i] = self._tree_prediction(
                                    test_x[:, feature_subsets[i]], trees[i])
        return predictions

    def predict(self, test_x):
        """Function returns the predictions of the random forest. Again we use 
        multiprocessing to split the task into chunks for a performance boost. 
        From fit() method we constructed n_jobs number of sub-forests. Here we 
        make the predictions for each of those sub-forests in parallel. 

        Parameters
        ----------
        test_x : numpy array of shape (n_test_obs, n_features) and type
        float.
        """
        with mp.Pool(processes=self.n_jobs) as pool:
            parallel_predictions = pool.starmap(self._forest_prediction, 
                                                [(test_x, 
                                                self.forests[i][0], 
                                                self.forests[i][1])
                                                for i in range(self.n_jobs)])
        all_predictions = np.vstack(parallel_predictions)
        final_predictions = np.empty(test_x.shape[0])
        for i in range(test_x.shape[0]):
            counts = np.bincount(all_predictions[:, i].astype(int), minlength=10)
            final_predictions[i] = np.argmax(counts)
        return final_predictions            